# Setup Env

In [1]:
! pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.0 MB/s eta 0:00:00


In [ ]:
from langchain_openai import ChatOpenAI

# Setup LLM

In [ ]:
def get_response(messages):
  llm = ChatOpenAI(model='gpt-4o', base_url="https://api.avalai.ir/v1", api_key="API-KEY")
  response = llm.invoke(messages)
  print("Request Sent!")

  return response.content

# Get Sample Sentences

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

import pandas as pd
import re


In [ ]:
# Load the CSV file containing Google Sheet names and links
sheets_info = pd.read_csv("/content/drive/MyDrive/human_sheet_links.csv", header=None, names=["sheet_name", "sheet_link"])

In [ ]:
import os
import csv

# Step 1: Download all Google Sheets as CSV files
os.makedirs("sheets_csv", exist_ok=True)
for idx, row in sheets_info.iterrows():
    if idx == 0: continue

    sheet_name = row["sheet_name"]
    sheet_link = row["sheet_link"]

    while True:
      try:
        # Open the Google Sheet
        sheet = gc.open_by_url(sheet_link)
        worksheet = sheet.get_worksheet(0)  # Assuming the first worksheet
        break
      except:
        continue


    # Export data to CSV
    csv_path = os.path.join("sheets_csv", f"{sheet_name}.csv")
    with open(csv_path, "w", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerows(worksheet.get_all_values())

In [ ]:
import math

In [ ]:
# Initialize the dictionary to store the word-to-pronunciations mapping
word_to_pronunciations = {}

# Function to check if a string contains only Persian letters
def is_persian_word(word):
    persian_pattern = re.compile(r"^[\u0600-\u06FF]+$")
    return bool(persian_pattern.match(word))

# Function to check if a string contains any English letters
def contains_english_letters(word):
    english_pattern = re.compile(r"[a-zA-Z]")
    return bool(english_pattern.search(word))

# Step 2: Process the downloaded CSV files
for csv_file in os.listdir("sheets_csv"):
    csv_path = os.path.join("sheets_csv", csv_file)
    sheet_data = pd.read_csv(csv_path, header=None).values.tolist()

    current_word = None

    for row in sheet_data:
        content = row[0]  # First column contains words and pronunciations
        sentence = row[1] if len(row) > 1 else ""  # Second column contains sentences

        if not (not isinstance(content, str) and math.isnan(content)) and not contains_english_letters(content):
            # Update the current word context
            current_word = content
            if current_word not in word_to_pronunciations:
                word_to_pronunciations[current_word] = {}
        elif current_word:
            # Add the sentence to the list of the corresponding pronunciation
            if content not in word_to_pronunciations[current_word]:
                word_to_pronunciations[current_word][content] = []
            if sentence and not (not isinstance(sentence, str) and math.isnan(sentence)):
                word_to_pronunciations[current_word][content].append(sentence)

# Output the updated dictionary
for word, data in word_to_pronunciations.items():
    print(f"{word}: {data}")


اعمال: {'?a?mAl': ['اعمال هر شخص مربوط ب خودش است', 'اعمال خوب به انسان در مسیر زندگی کمک میکند', 'باید مراقب اعمال خود باشیم', 'اعمال بد عواقب بدی هم دارد', 'هرکسی از قوانین پیروی نکند اعمال قانون میشود'], '?e?mAl': ['پلیس کسی که تخلف کند را اعمال قانون میکند', 'هر شخصی در زندگی خود باید قوانین را اعمال کند', 'هر شخصی برای خود قوانین به خصوصی دارد ک اعمال میکند', 'اعمال قوانین توسط هرشخص برای زندگی در جامعه واجب است']}
کرم: {'kerem': ['کرم ضد آفتاب برای پوست در مقابل نور آفتاب مفید است', 'کرم میتواند فواید زیادی برای پوست داشته باشد', 'کرم دست و صورت باهم متفاوت هستند', 'کرم میتواند بوهای متفاوتی داشته باشد', 'بخشش از کرم و بزرگی است'], 'karam': ['هر چه بخشش و کرم بیشتری داشته باشی ارزش بالاتری داری', 'بخشش او از سخاوت و کرم او سرچشمه می\u200cگیرد ', 'کرم و شکوه آن مرد همه را مجذوب می\u200cکرد ', 'کرم صفتی است که به افراد گرامی نسبت داده می\u200cشود ', 'رنگ کروم براق است'], 'korom': ['رینگ ماشین من کروم است', 'کروم رنگ خاصی است', 'کروم رنگ زیبایی است', 'من اسپری رنگ کروم خریدم', 'کرم 

In [ ]:
# Step 3: Update word_to_pronunciations with data from the helper CSV
helper_csv_path = "commonvoice_test_dict - Ambiguous Words.csv"  # Path to the helper CSV
helper_data = pd.read_csv(helper_csv_path)

for _, row in helper_data.iterrows():
    grapheme = row["grapheme"]
    phoneme = row["phoneme"]
    polyphone = row["polyphone"]
    the_right_phone = row["the right phone"]

    if grapheme in word_to_pronunciations:
        pronunciations = word_to_pronunciations[grapheme]
        if the_right_phone in pronunciations and not pronunciations[the_right_phone]:
            # Add the sample sentence if available
            if pd.notna(phoneme):
                pronunciations[the_right_phone].append(phoneme)

# Output the updated dictionary
for word, data in word_to_pronunciations.items():
    print(f"{word}: {data}")

اعمال: {'?a?mAl': ['اعمال هر شخص مربوط ب خودش است', 'اعمال خوب به انسان در مسیر زندگی کمک میکند', 'باید مراقب اعمال خود باشیم', 'اعمال بد عواقب بدی هم دارد', 'هرکسی از قوانین پیروی نکند اعمال قانون میشود'], '?e?mAl': ['پلیس کسی که تخلف کند را اعمال قانون میکند', 'هر شخصی در زندگی خود باید قوانین را اعمال کند', 'هر شخصی برای خود قوانین به خصوصی دارد ک اعمال میکند', 'اعمال قوانین توسط هرشخص برای زندگی در جامعه واجب است']}
کرم: {'kerem': ['کرم ضد آفتاب برای پوست در مقابل نور آفتاب مفید است', 'کرم میتواند فواید زیادی برای پوست داشته باشد', 'کرم دست و صورت باهم متفاوت هستند', 'کرم میتواند بوهای متفاوتی داشته باشد', 'بخشش از کرم و بزرگی است'], 'karam': ['هر چه بخشش و کرم بیشتری داشته باشی ارزش بالاتری داری', 'بخشش او از سخاوت و کرم او سرچشمه می\u200cگیرد ', 'کرم و شکوه آن مرد همه را مجذوب می\u200cکرد ', 'کرم صفتی است که به افراد گرامی نسبت داده می\u200cشود ', 'رنگ کروم براق است'], 'korom': ['رینگ ماشین من کروم است', 'کروم رنگ خاصی است', 'کروم رنگ زیبایی است', 'من اسپری رنگ کروم خریدم', 'کرم 

# Generate Sample Sentences

In [ ]:
# Step 4: Generate additional sentences using an LLM
output_csv_path = "/content/drive/MyDrive/openai_generated_sentences.csv"
processed_pairs_path = "/content/drive/MyDrive/openai_processed_pairs.csv"

In [ ]:
existing_data = set()
processed_pairs = set()

# Load existing data if the file exists
if os.path.exists(output_csv_path):
    with open(output_csv_path, "r", encoding="utf-8") as f:
        reader = csv.reader(f)
        next(reader)  # Skip the header
        for row in reader:
            existing_data.add((row[0], row[1], row[2]))

# Load processed pairs if the file exists
if os.path.exists(processed_pairs_path):
    with open(processed_pairs_path, "r", encoding="utf-8") as f:
        reader = csv.reader(f)
        for row in reader:
            processed_pairs.add((row[0], row[1]))

with open(output_csv_path, "a", encoding="utf-8", newline="") as output_file, \
     open(processed_pairs_path, "a", encoding="utf-8", newline="") as pairs_file:

    writer = csv.writer(output_file)
    pairs_writer = csv.writer(pairs_file)

    if not existing_data:
        writer.writerow(["persian word", "pronunciation", "sample sentence"])

    for word, pronunciations in word_to_pronunciations.items():
        for pronunciation, sentences in pronunciations.items():
            if (word, pronunciation) in processed_pairs:
                continue

            if sentences:
                # Generate new sentences using the LLM
                messages = [
                    {
                        'role': 'system',
                        'content': 'You are a helpful assistant skilled in generating sentences in Persian, especially for words with multiple meanings and pronunciations. Your responses should demonstrate appropriate use of homographs and provide context for each pronunciation.'
                    },
                    {
                        'role': 'user',
                        'content': f'''Your task is to generate sentences for a Persian homograph word that has multiple pronunciations and meanings. The word is "{word}" with pronunciations "{list(pronunciations.keys())}". Here are some sample sentence for the pronunciation "{pronunciation}": "{', '.join(sentences)}". Please generate 50 additional sentences with "{word}" with this pronunciation. Note that these are only sample sentences, you must generate different and diverse sentences with this meaning. Feel free to append prefixes/suffixes to the word as long as you include the word itself.
                         Please return each output sentence within brackets.'''
                    }
                ]
                print(f"Prompting for {word}, {pronunciation}")
                response = get_response(messages)
                print(response)

                # Extract sentences within brackets using regex
                generated_sentences = re.findall(r'\[(.*?)\]', response)

                # Write generated sentences to the CSV file
                for sentence in generated_sentences:
                    if (word, pronunciation, sentence) not in existing_data:
                        writer.writerow([word, pronunciation, sentence])
                        existing_data.add((word, pronunciation, sentence))
                    else:
                      print("Duplicate!")

                # Mark the pair as processed
                pairs_writer.writerow([word, pronunciation])
                processed_pairs.add((word, pronunciation))

print(f"Generated sentences have been saved to {output_csv_path}")

Streaming output truncated to the last 5000 lines.
28. [پدربزرگم در طول سال‌ها شخصیت مهربان و دلسوزی پرورد.]
29. [این ورزشگاه محلی برای پرورش استعدادهای ورزشی جدید پرورد.]
30. [او در طول سفرهایش تجربیات ارزشمندی پرورد که در زندگی‌اش مؤثر بودند.]
31. [با تمرین و ممارست، او توانایی‌های خود در موسیقی را پرورد.]
32. [مادر همیشه دستور پخت‌های خاصی پرورد که طعم‌های بی‌نظیری داشتند.]
33. [این شرکت با استفاده از فناوری‌های پیشرفته، محصولات رقابتی پرورد.]
34. [در این محیط آموزشی، مهارت‌های ارتباطی دانش‌آموزان به خوبی پرورده شد.]
35. [نویسنده در طول سال‌ها سبک خاصی در نوشتن پرورد که مخاطبان زیادی را جذب کرد.]
36. [این گروه موسیقی با تلاش و تمرین فراوان، قطعات بی‌نظیری پرورد.]
37. [در این آزمایشگاه، مواد جدیدی برای استفاده در صنعت ساختمان پروردند.]
38. [با تلاش مستمر، او توانست روابط کاری موفقی پرورد.]
39. [این شهر کوچک به مرور زمان به مرکز فرهنگی بزرگی پرورد.]
40. [پژوهشگران در این مرکز تحقیقاتی، روش‌های نوینی برای درمان بیماری‌ها پروردند.]
41. [در این باغ وحش، گونه‌های نادری از حیوانات پرورده ش